1) Необходимо прочитать файл cinema.json, используя библиотеку json. https://docs.python.org/3/library/json.html

Создать новые файлы cinema2.json и cinema2.csv, в которых для каждого кинотеатра будут отображены только следующие поля из исходного файла: 

* ID, 
* Краткое наименование (CommonName), 
* Адрес (текстовой строкой, только улица-дом, без индекса и района), 
* Сайт, 
* Рабочие часы по дням недели (в формате словаря "День недели: часы работы"), 
* Долгота, 
* Широта.

In [26]:
import json
import pprint
with open("cinema.json", "rt") as file:
    data=json.load(file)
cinema_list=[]
for row in data:
    cinema_list.append({
    "ID" : row['global_id'],
    "CommonName" : row['CommonName'],
    "Address" : row['ObjectAddress'][0]['Address'],
    "WebSite" : row['WebSite'],
    "WorkingHours" : row['WorkingHours'],
    "Dolgota" : row['geoData']['coordinates'][0][0],
    "Shirota" : row['geoData']['coordinates'][0][1]})
pprint.pprint(cinema_list)

[{'Address': 'улица Кедрова, дом 14, корпус 3',
  'CommonName': 'Кинотеатр «Салют»',
  'Dolgota': 37.571298748323,
  'ID': 477012208,
  'Shirota': 55.682806921335,
  'WebSite': 'mos-kino.ru',
  'WorkingHours': [{'DayWeek': 'понедельник', 'WorkHours': '09:00-22:00'},
                   {'DayWeek': 'вторник', 'WorkHours': '09:00-22:00'},
                   {'DayWeek': 'среда', 'WorkHours': '09:00-22:00'},
                   {'DayWeek': 'четверг', 'WorkHours': '09:00-22:00'},
                   {'DayWeek': 'пятница', 'WorkHours': '09:00-22:00'},
                   {'DayWeek': 'суббота', 'WorkHours': '09:00-22:00'},
                   {'DayWeek': 'воскресенье', 'WorkHours': '09:00-22:00'}]},
 {'Address': 'улица Кухмистерова, дом 4',
  'CommonName': 'Кинотеатр «Тула»',
  'Dolgota': 37.718514574853,
  'ID': 477012215,
  'Shirota': 55.685873427249,
  'WebSite': 'mos-kino.ru',
  'WorkingHours': [{'DayWeek': 'понедельник', 'WorkHours': '10:00-22:00'},
                   {'DayWeek': 'вторник', '

2) Присутствуют два csv-файла. В одном ID и координаты узлов графа (points.csv), в другом - координаты ребер графа, полученные с географической карты (polylines.csv). Необходимо к координатам из ребер графа привязать соответствующие им узлы графа (смотреть по близости их координат) и создать csv файл с полями "ID начального узла ребра" и "ID конечного узла ребра". 

**Важно**: координаты узлов могут минимально расходиться с координатами начала и концов ребер, поэтому свести по точному соответствию не получится.

Справка по работе с csv: https://docs.python.org/3/library/csv.html

In [62]:
import csv
lst3 = []
with open('polylines.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=';')
    for row in csv_reader:
        if row != []:
            lst3.append(row[0].replace('"', '').replace('),(', ');(').split(";"))

with open('polylines_remastered.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    
    for i in lst3:
        writer.writerow(i)

with open('polylines.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=';')
    for row in csv_reader:
        if row != []:
            lst3.append(row[0].replace('"', '').replace('),(', ');(').split(";"))

with open('polylines_remastered.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    
    for i in lst3:
        writer.writerow(i)
points = []
with open('points.csv','r') as csv_file:
    csv_reader =csv.reader(csv_file)
    current_id=1
    for row in csv_reader:
        if len(row) > 0:
            points.append({
                'id' : current_id,
                'lat': float(row[0]),
                'lon': float(row[1])
            })
            current_id+=1
lst_full = []
with open('polylines_remastered.csv','r',newline='') as csvfile:
    reader = csv.reader (csvfile,delimiter=';')
    counter=0
    for row in reader :
        lst=[]
        for el in row:
            lst.append(point_remaster(el,points))
        print(counter,row)
        counter+=1
        lst_full.append([i for i in lst if i!= None])
def point_remaster(point,dicti):
    point_tuple = (
       float(point.split(',')[0][1:]),
       float(point.split(',')[1][:-1])
            )
    for row in dicti:
        if -0.0005 <= point_tuple[0]-row['lat'] <= 0.0005 and -0.0005<= point_tuple[1] - row['lon'] <= 0.0005:
              return row['id']
final_list = []
current_id = 1
for i, e in enumerate(lst_full,1):
    for ind,el in enumerate(e[:-1]):
        final_list.append({
            "id" : current_id,
            "graph_id" : i,
            "point_from" : el,
            "point_to": e[ind+1]
        })
        
print(final_list) 

[{'id': 1, 'graph_id': 1, 'point_from': 86, 'point_to': 184}, {'id': 1, 'graph_id': 1, 'point_from': 184, 'point_to': 434}, {'id': 1, 'graph_id': 1, 'point_from': 434, 'point_to': 198}, {'id': 1, 'graph_id': 1, 'point_from': 198, 'point_to': 199}, {'id': 1, 'graph_id': 1, 'point_from': 199, 'point_to': 200}, {'id': 1, 'graph_id': 1, 'point_from': 200, 'point_to': 197}, {'id': 1, 'graph_id': 1, 'point_from': 197, 'point_to': 196}, {'id': 1, 'graph_id': 1, 'point_from': 196, 'point_to': 195}, {'id': 1, 'graph_id': 1, 'point_from': 195, 'point_to': 194}, {'id': 1, 'graph_id': 1, 'point_from': 194, 'point_to': 23}, {'id': 1, 'graph_id': 1, 'point_from': 23, 'point_to': 298}, {'id': 1, 'graph_id': 1, 'point_from': 298, 'point_to': 52}, {'id': 1, 'graph_id': 1, 'point_from': 52, 'point_to': 540}, {'id': 1, 'graph_id': 1, 'point_from': 540, 'point_to': 245}, {'id': 1, 'graph_id': 1, 'point_from': 245, 'point_to': 174}, {'id': 1, 'graph_id': 1, 'point_from': 174, 'point_to': 155}, {'id': 1, 'g